In [51]:
!pip install geopandas
!pip install contextily

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/opt/conda/envs/r/bin/pip", line 6, in <module>
    from pip._internal.cli.main import main
  File "/opt/conda/envs/r/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/opt/conda/envs/r/lib/python3.10/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/opt/conda/envs/r/lib/python3.10/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/opt/conda/envs/r/lib/python3.10/site-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/opt/conda/envs/r/lib/python3.10/site-packages/pip/_intern

In [52]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [53]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import pandas as pd
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [54]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [55]:
iso3 = "UGA"  ##GHA
file_path = f"/projects/my-public-bucket/AOIs/vero_1deg_tileindex/tileindex_{iso3}.csv"
tileindex_df = pd.read_csv(file_path)
# Display the first few rows of the DataFrame
#print(tileindex_df.head())

In [56]:
json_files = tileindex_df['s3path']
AOIs = json_files
print(len(AOIs))
aoi_name = os.path.basename(AOIs[0]).split('.')[0]
aoi_name

34


'tile_num_37710'

In [57]:
AOIs[0]

'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_37710.geojson'

In [50]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32vcpu-64gb",  #"maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result



Thanks, I'll run the GEDI L4A subsetter for tile_num_37710!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37711!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37712!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37887!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37888!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37889!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37890!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37891!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_37892!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_38067!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_38068!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_38069!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_38070!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_38071!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_38072!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_38247!!
Thanks, I'll run the GED

In [ ]:
## if jobs fail re-run them using the cell below

In [42]:
PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
## if necessary ## re-run failed subsets
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_16276.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_32125.geojson'
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_37711.geojson'
inputs = dict(
    aoi=each_aoi,
    doi="L4A",
    lat="lat_lowestmode",
    lon="lon_lowestmode",
    beams="all",
    columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
    query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.9.0",
    queue="maap-dps-worker-32vcpu-64gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'f9076ddd-7b37-4fe4-9542-4dc298681b8c'

In [58]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


1820


In [59]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2025/01/08"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
#            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
            print(f"Copying {fname}...")
            subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_16276_L4A.gpkg...
Copying tile_num_16457_L4A.gpkg...
Copying tile_num_16275_L4A.gpkg...
Copying tile_num_16456_L4A.gpkg...
Copying tile_num_16814_L4A.gpkg...
Copying tile_num_16994_L4A.gpkg...
Copying tile_num_17175_L4A.gpkg...
Copying tile_num_16455_L4A.gpkg...
Copying tile_num_16995_L4A.gpkg...
Copying tile_num_16638_L4A.gpkg...
Copying tile_num_16637_L4A.gpkg...
Copying tile_num_16996_L4A.gpkg...
Copying tile_num_17356_L4A.gpkg...
Copying tile_num_16636_L4A.gpkg...
Copying tile_num_17176_L4A.gpkg...
Copying tile_num_16635_L4A.gpkg...
Copying tile_num_16997_L4A.gpkg...
Copying tile_num_16817_L4A.gpkg...
Copying tile_num_16816_L4A.gpkg...
Copying tile_num_17353_L4A.gpkg...
Copying tile_num_17355_L4A.gpkg...
Copying tile_num_16815_L4A.gpkg...
Copying tile_num_32124_L4A.gpkg...
Copying tile_num_32123_L4A.gpkg...
Copying tile_num_31944_L4A.gpkg...
Copying tile_num_31945_L4A.gpkg...
Copying tile_num_31943_L4A.gpkg...
Copying tile_num_31940_L4A.gpkg...
Copying tile_num_319

In [60]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


1823
